In [1]:
import os
import sys
import traceback
from collections import Counter,OrderedDict

import numpy as np
from hops import hdfs
from hops import pandas_helper as pandas

import utils

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
606,application_1615491195715_0807,pyspark,idle,Link,Link


SparkSession available as 'spark'.


In [17]:


args_full=utils.load_arguments(sys.argv)
args=args_full[utils.KEY_FILTER_DIAMOND]

OUTPUT_DATASET=args_full[utils.OUTPUT_DATASET]
INPUT_ROOT_PATH=args_full[utils.INPUT_ROOT_PATH]
RUN_FOLDER=args_full[utils.RUN_FOLDER]
WORK_PATH=os.path.join(OUTPUT_DATASET, RUN_FOLDER)

REPORT_FILE=args['REPORT_FILE']
FILTERED_PREFIX='_filtered'
# check of input and output root override
if args_full.get(utils.INPUT_OVERRIDE):
    INPUT_ROOT=args_full.get(utils.INPUT_OVERRIDE)
else :
    INPUT_ROOT=os.path.join(WORK_PATH,args['INPUT_ROOT'])
if args_full.get(utils.OUTPUT_OVERRIDE):
    OUTPUT_ROOT=args_full.get(utils.OUTPUT_OVERRIDE)
else:
    OUTPUT_ROOT=os.path.join(WORK_PATH,args['OUTPUT_ROOT'])




In [18]:
all_files=utils.load_file_names(INPUT_ROOT)

print('Number of input files at input path', len(all_files) , INPUT_ROOT)

input files  9759

In [2]:
""" header variables """
HDR_FILE_NAME="File Name"
HDR_SAMPLE_NAME="Sample Name"
HDR_TOTAL_READS="Total Reads"
HDR_UNIQUE_READS="Unique Reads"
HDR_UNIQUE_PAIR_ENDS="Unique pairs"
HDR_HPV_COUNT_UNIQUE="HPV Type-Reads Unique Counts"
HDR_HPV_COUNT_EXCLUSIVE="HPV Type-Reads Exclusive Counts"
HDR_HPV_TYPES_NONOVERLAP="HPV Type-Reads Most Abundant"
HDR_HPV_TYPE_COV_PAIR="HPV Coverage-Pair"
HDR_COV_PER_SUBTYPE="Coverage per SubjectType"
HDR_POVSITIVE="Positive"

# Dict (header, rowData)
headerRowData=OrderedDict() # type: Dict[str, str]
# initliase headers in following order
headerRowData[HDR_FILE_NAME]=""
headerRowData[HDR_SAMPLE_NAME]=""
headerRowData[HDR_TOTAL_READS]=""
headerRowData[HDR_UNIQUE_READS]=""
headerRowData[HDR_UNIQUE_PAIR_ENDS]=""
headerRowData[HDR_HPV_COUNT_UNIQUE]=""
headerRowData[HDR_HPV_COUNT_EXCLUSIVE]=""
headerRowData[HDR_HPV_TYPES_NONOVERLAP]=""
headerRowData[HDR_HPV_TYPE_COV_PAIR]=""
headerRowData[HDR_COV_PER_SUBTYPE]=""
headerRowData[HDR_POVSITIVE]=""





def table_collect_stats(file_path):
    """
    Table to create statistics on 1 diamond report file.
    The output is used to write into a file
    :param String:diamond report hdfs file_path :
    :return: List: with each item corresponding to 1 row for report
    """

    subjects_reads_exclusive_full=''
    subjects_reads_uniqueNonOverlap_full=''
    subjects_reads_unique_full=''
    subjects_cov_len=''
    subject_unique_reads={} #  {subjectType: uniqueReadsCount}
    coverage_per_sub_type={} # {subjectType: totalCoverageLen}
    is_positive=False
    subject_total_cov_len=''

    file_path=hdfs.get_plain_path(file_path)
    file_name=os.path.basename(file_path)

    print('Processing file: ',  file_path)
    try :
        if FILTERED_PREFIX in file_path: # old job of filtering diamond positives added this prefix. Otherwise not needed
            df=pandas.read_csv(file_path, sep='\t', names=["Query", "Subject", "Percentage", "Alignment",'Mismatch','Gap','Start','End','SA','EA','EV','BitScore'],header=1)
        else :
            df=pandas.read_csv(file_path, sep='\t', names=["Query", "Subject", "Percentage", "Alignment",'Mismatch','Gap','Start','End','SA','EA','EV','BitScore'],header=None)

        totalRows=df.shape[0]
        queries=df.Query
        subjects=df.Subject
        unique_queries=queries.unique()
        unique_queries_set=set(unique_queries)
        uniqueQueries_set_reducing=set(unique_queries.copy()) # used in calculate non overlapping reads
        unique_count=len(unique_queries)  # unique reads
         # subject types
        subjects_filtered = get_subject_types(subjects)
        df['SubjectType']=subjects_filtered
        # protein type
        protein_filtered=[ x.split('|')[1].split('.')[1] for x in subjects ]
        df['ProteinType']=protein_filtered
        # unique subject types
        groupedQuerySubject=df[['Query','SubjectType']].groupby('SubjectType',sort=False).count().sort_values(by=['Query'],ascending=False)
        # protein type-subject type grouped ordered by highest reads first
        groupedProteinSubject=df[['Query','SubjectType','ProteinType']].groupby(['SubjectType','ProteinType']).count().sort_values(by=['Query'],ascending=False)

        #  Iterate DF with query, subject type and protein type
        for index, row in groupedQuerySubject.iterrows():
            # get all reads for current subject type
            current_reads=df[df.SubjectType == index].Query
            # unique reads for current HPV type=intersection of total uniques and current reads
            unique_reads_count=len(unique_queries_set.intersection(current_reads))
            subjects_reads_unique_full = subjects_reads_unique_full+index+'='+str(unique_reads_count)+';' # construct string
             # get exclusive HPV counts for subject type
            unique_reads_count_exclusive=exclusive_reads_count(df,index)
            subjects_reads_exclusive_full = subjects_reads_exclusive_full+index+'='+str(unique_reads_count_exclusive)+';' # construct string
            subject_unique_reads[index] = unique_reads_count # add to dict unique reads count
            '''
            !! For unique non overlapping (most adundant) HPV counts. !!
            !! Shared reads are count only for the one with most reads !!
            This is the unique reads for subject type by excluding the reads
            counted for previous subject types in the sort order of results.
            1. get unique reads by intersection of total unique reads set after the reads of previous subject type are counted
            2. remove these reads from the total unique set as they are counted for current subject type
            '''
            unique_currentReads=uniqueQueries_set_reducing.intersection(current_reads)
            uniqueQueries_set_reducing=uniqueQueries_set_reducing.difference(unique_currentReads)
            subjects_reads_uniqueNonOverlap_full = subjects_reads_uniqueNonOverlap_full+index+'='+str(len(unique_currentReads))+';'


        # Coverage length per subject type
        for index,row in groupedProteinSubject.iterrows():
            subjectType = index[0]
            protein =  index[1]
            alignmentDf = df[(df.SubjectType ==  subjectType ) & (df.ProteinType == protein )]
            alignmentPositions=alignmentDf[['SA','EA']].values.tolist()
            coverageLen_current=count_merged_intervals(alignmentPositions)
            subjects_cov_len=subjects_cov_len+index[0]+'|'+index[1]+'='+str(coverageLen_current)+';'
            covLenPerSub=coverage_per_sub_type.get(subjectType)
            if covLenPerSub:
                coverage_per_sub_type[subjectType]=covLenPerSub+coverageLen_current
            else :
                coverage_per_sub_type[subjectType]=coverageLen_current


         # total cov len per subject type
        for subType,covLen in coverage_per_sub_type.items():
            subject_total_cov_len = subject_total_cov_len+subType+'='+str(covLen)+';'
            # check for positive
            reads=subject_unique_reads.get(subType)
            if not is_positive:
                is_positive=check_positive(reads,covLen)


        # paired reads count
        querySubject=df[['Query','SubjectType']].sort_values(by=['Query'])
        queries=querySubject.Query
        subjects=querySubject.SubjectType
        r1=[]
        r2=[]
        for count,item in enumerate(queries):
            seq=item.split('/')[0]
            if '/1' in item:
                r1.append((seq,subjects[count]))
            elif '/2':
                r2.append((seq,subjects[count]))

        r1_dict = dict(Counter(r1))
        r2_dict = dict(Counter(r2))
        pairs=0 # number of unique pairs
        for r1_item in r1_dict.items():
            r1_key=r1_item[0]
            r1_value=r1_item[1]
            if r1_key in r2_dict:
                r2_value=r2_dict.get(r1_key)
                pairs+=min([r1_value,r2_value])

        # get original sample name
        sample=os.path.splitext(file_name)[0].replace('_NH_unmapped','').replace('Hpv_trim_','').replace('_filtered','')


        # add values into header-row map
        headerRowData[HDR_FILE_NAME]=file_name
        headerRowData[HDR_SAMPLE_NAME]=sample
        headerRowData[HDR_TOTAL_READS]=str(totalRows)
        headerRowData[HDR_UNIQUE_READS]=str(unique_count)
        headerRowData[HDR_UNIQUE_PAIR_ENDS]=str(pairs)
        headerRowData[HDR_HPV_COUNT_UNIQUE]=subjects_reads_unique_full
        headerRowData[HDR_HPV_COUNT_EXCLUSIVE]=subjects_reads_exclusive_full
        headerRowData[HDR_HPV_TYPES_NONOVERLAP]=subjects_reads_uniqueNonOverlap_full
        headerRowData[HDR_HPV_TYPE_COV_PAIR]=subjects_cov_len
        headerRowData[HDR_COV_PER_SUBTYPE]=subject_total_cov_len
        headerRowData[HDR_POVSITIVE]=str(is_positive)

        return str.join('\t', headerRowData.values()) # return row as single string
    except Exception as e:
        print(e)
        traceback.print_exc()
        return None





def exclusive_reads_count(df,subject_type):
    """
    Calculate unique exclusive reads for HPV type
    :param df: DF with subject type and queries
    :param subject_type: HPY type
    :return: Integer: exclusive unique reads count
    """
    current_reads=set(df[df.SubjectType == subject_type].Query.unique())
    other_types_reads=set(df[df.SubjectType != subject_type].Query.unique())
    unique_reads_count=len(current_reads.difference(other_types_reads))
    return unique_reads_count
        

def count_merged_intervals(intervals, sort=True):
    '''Gets a list of intervals (should be sorted or marked to be sorted with sort=True), 
       merges them when overlapping and then calculate the sum (including limits).
       Original list: [4, 8], [10, 12], [1, 5]
       Sorted and merged: [1, 8], [10, 12]
       Returns: 11
    '''
    intervals = np.array(intervals)
    if sort:
        intervals.sort(axis=0)
    starts = intervals[:, 0]
    ends = np.maximum.accumulate(intervals[:, 1])
    valid = np.zeros(len(intervals) + 1, dtype=np.bool)
    valid[0] = True
    valid[-1] = True
    valid[1:-1] = starts[1:] >= ends[:-1]
    merged = np.vstack((starts[:][valid[:-1]], ends[:][valid[1:]])).T
    # resultado2 = np.zeros(len(merged), dtype=np.int)
    resultado2 = merged[:, 1] - merged[:, 0] + 1
    return np.sum(resultado2)


def check_positive(reads,coverage,threshold_unique_count=10,threshold_coverage=230):
    """
    Flag sample as positive or negative based on thresholds
    :param reads: Reads for given subject type
    :param coverage: Coverage length for given subjet type
    :param threshold_unique_count: threshold for reads
    :param threshold_coverage: threshold for coverage length
    :return: True if positive
    """
    if reads >= threshold_unique_count and coverage >= threshold_coverage:
        return True
    return False


def get_subject_types(subjects):
    """
    Split Subjects column into corresponding subject types
    :param subjects: List
    :return: subjects_filtered:List
    """
    splt_char='-'
    K=2 # index for splitted array
    subjects_filtered=[]
    for item in subjects:
        sub_grp=item.split('|')[-2]
        splits_arr = sub_grp.split(splt_char)
        if len(splits_arr)==3: # incase of exceptions e.g. HPV-mSK152
            sub = splt_char.join(splits_arr[:K])
        else :
            sub = splits_arr[0]
        subjects_filtered.append(sub)
    
    return subjects_filtered





NameError: name 'OrderedDict' is not defined

In [24]:
""" for testing"""
#table2('hdfs:///Projects/TCGA_viruses/Diamond/raw/Hpv_trim_008d4eb5-91d6-4c69-b24b-afe15ff18c17_gdc_realn_rehead_NH_unmapped')
#table2('hdfs:///Projects/TCGA_viruses/Diamond/raw/Hpv_trim_004660a1-aaed-4619-87a0-e8260f09128a_gdc_realn_rehead_NH_unmapped')
#table2('hdfs:///Projects/TCGA_viruses/Diamond/raw/Hpv_trim_09f31b55-df37-4673-8a8c-bde9bc8780c4_gdc_realn_rehead_NH_unmapped')

#all_files=['hdfs:///Projects/TCGA_viruses/Diamond/raw/Hpv_trim_004660a1-aaed-4619-87a0-e8260f09128a_gdc_realn_rehead_NH_unmapped']

In [29]:
rdd=sc.parallelize(all_files)



In [37]:
outputLines=rdd.map(table_collect_stats).collect()
len(outputLines)

9759

#### Write into file

In [38]:
def writeFile(outputLinesList):

    with open(REPORT_FILE, 'w') as f:
        headers=str.join('\t',headerRowData.keys())
        f.write(headers)
        f.write('\n')
        for i in outputLinesList:
            f.write(i+'\n')


    hdfs.copy_to_hdfs(REPORT_FILE,OUTPUT_ROOT,overwrite=True)


writeFile(outputLines)

Started copying local path report_diamond_raw_uniqueHpv.txt to hdfs path hdfs://rpc.namenode.service.consul:8020/Projects/TCGA_viruses/Temp//report_diamond_raw_uniqueHpv.txt

Finished copying

'Hpv_trim_0047afe9-2225-4a5d-8cfa-234b1f8ba78c_gdc_realn_rehead_NH_unmapped\t0047afe9-2225-4a5d-8cfa-234b1f8ba78c_gdc_realn_rehead\t15\t13\t0\tHPV170=3;HPV-mICB2=2;HPV-mSK065=2;HPV173=2;HPV-mSK037=2;HPV51=1;HPV-mSK100=1;HPV112=1;HPV-mw18c07=1;\tHPV170=3;HPV-mICB2=2;HPV-mSK065=2;HPV173=0;HPV-mSK037=2;HPV51=1;HPV-mSK100=1;HPV112=1;HPV-mw18c07=1;\tHPV170|E1=21;HPV-mICB2|L2=21;HPV-mSK037|E1=24;HPV-mSK065|L1=17;HPV173|L1=17;HPV-mSK100|E1^E4=21;HPV-mw18c07|L2=16;HPV112|L2=18;HPV51|E1=16;\tHPV170=21;HPV-mICB2=21;HPV-mSK037=24;HPV-mSK065=17;HPV173=17;HPV-mSK100=21;HPV-mw18c07=16;HPV112=18;HPV51=16;\tFalse'